<a href="https://colab.research.google.com/github/wallisonferreira/github_data/blob/master/extract_commits_from_repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#####Invite people for the party

In [1]:
import json
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '/content/drive/MyDrive/JupyterNotebooks/JupyterEnviroments/JupyterConfig')
import config

#####Creating a session

In [31]:
gh_api = 'https://api.github.com'
gh_session = requests.Session()
gh_session.auth = (
    config.github_client['name'],
    config.github_client['access_token']
)

#####Obtain repos information

In [15]:
url = gh_api + "/repos/apache/spark/commits"
commits = gh_session.get(url=url)
commits_json = commits.json()

In [36]:
def commits_of_repo_github(repo, owner, api):
    commits = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/commits?page={}&per_page=100'.format(owner, repo, i)
        commit_pg = gh_session.get(url = url)
        commit_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in commit_pg.json()]    
        commit_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in commit_pg_list]
        commits = commits + commit_pg_list
        if 'Link' in commit_pg.headers:
            if 'rel="next"' not in commit_pg.headers['Link']:
                next = False
        i = i + 1
    return commits

In [37]:
def create_commits_df(repo, owner, api):
    commits_list = commits_of_repo_github(repo, owner, api)
    return json_normalize(commits_list)

In [ ]:
commits_df = create_commits_df("spark", "apache", gh_api)

In [42]:
commits_df.to_csv('/content/drive/MyDrive/GithubAnalysis/ApacheSpark/commits.csv')

In [63]:
list(commits_df.columns)[0:10]

['sha',
 'node_id',
 'url',
 'html_url',
 'comments_url',
 'parents',
 'repo_name',
 'owner',
 'commit.author.name',
 'commit.author.email']

In [47]:
commits_df.head()

,sha,node_id,url,html_url,comments_url,parents,repo_name,owner,commit.author.name,commit.author.email,commit.author.date,commit.committer.name,commit.committer.email,commit.committer.date,commit.message,commit.tree.sha,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,commit.verification.payload,author.login,author.id,author.node_id,author.avatar_url,author.gravatar_id,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.id,committer.node_id,committer.avatar_url,committer.gravatar_id,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin,committer,author
0,b2a2b5d8206b7c09b180b8b6363f73c6c3fdb1d8,MDY6Q29tbWl0MTcxNjU2NTg6YjJhMmI1ZDgyMDZiN2MwOW...,https://api.github.com/repos/apache/spark/comm...,https://github.com/apache/spark/commit/b2a2b5d...,https://api.github.com/repos/apache/spark/comm...,[{'sha': 'a9345a05911b8620387b36023ddc6525ed1a...,spark,apache,Liang-Chi Hsieh,viirya@gmail.com,2021-04-23T20:56:53Z,Liang-Chi Hsieh,viirya@gmail.com,2021-04-23T20:56:53Z,[SPARK-34297][SQL][SS] Add metrics for data lo...,1742b3a2568af2fabdf6b187d207697d29753fb8,https://api.github.com/repos/apache/spark/git/...,https://api.github.com/repos/apache/spark/git/...,0,False,unsigned,None,None,viirya,68855.0,MDQ6VXNlcjY4ODU1,https://avatars.githubusercontent.com/u/68855?v=4,,https://api.github.com/users/viirya,https://github.com/viirya,https://api.github.com/users/viirya/followers,https://api.github.com/users/viirya/following{...,https://api.github.com/users/viirya/gists{/gis...,https://api.github.com/users/viirya/starred{/o...,https://api.github.com/users/viirya/subscriptions,https://api.github.com/users/viirya/orgs,https://api.github.com/users/viirya/repos,https://api.github.com/users/viirya/events{/pr...,https://api.github.com/users/viirya/received_e...,User,False,viirya,68855.0,MDQ6VXNlcjY4ODU1,https://avatars.githubusercontent.com/u/68855?v=4,,https://api.github.com/users/viirya,https://github.com/viirya,https://api.github.com/users/viirya/followers,https://api.github.com/users/viirya/following{...,https://api.github.com/users/viirya/gists{/gis...,https://api.github.com/users/viirya/starred{/o...,https://api.github.com/users/viirya/subscriptions,https://api.github.com/users/viirya/orgs,https://api.github.com/users/viirya/repos,https://api.github.com/users/viirya/events{/pr...,https://api.github.com/users/viirya/received_e...,User,False,NaN,NaN
1,a9345a05911b8620387b36023ddc6525ed1a508c,MDY6Q29tbWl0MTcxNjU2NTg6YTkzNDVhMDU5MTFiODYyMD...,https://api.github.com/repos/apache/spark/comm...,https://github.com/apache/spark/commit/a9345a0...,https://api.github.com/repos/apache/spark/comm...,[{'sha': 'e503b9c462676c381ba51ad48edd968d6e18...,spark,apache,Wenchen Fan,wenchen@databricks.com,2021-04-23T16:21:43Z,Max Gekk,max.gekk@gmail.com,2021-04-23T16:21:43Z,[SPARK-35204][SQL] CatalystTypeConverters of d...,6ad693a2a6d595867bac6721dc8b699d14acf5ff,https://api.github.com/repos/apache/spark/git/...,https://api.github.com/repos/apache/spark/git/...,0,False,unsigned,None,None,cloud-fan,3182036.0,MDQ6VXNlcjMxODIwMzY=,https://avatars.githubusercontent.com/u/318203...,,https://api.github.com/users/cloud-fan,https://github.com/cloud-fan,https://api.github.com/users/cloud-fan/followers,https://api.github.com/users/cloud-fan/followi...,https://api.github.com/users/cloud-fan/gists{/...,https://api.github.com/users/cloud-fan/starred...,https://api.github.com/users/cloud-fan/subscri...,https://api.github.com/users/cloud-fan/orgs,https://api.

In [50]:
commits_df.shape

(29895, 61)